## 데이터 작업하기

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 공개 데이터셋에서 학습 데이터 내려받기
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터 내려받기
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|███████████████████████████████████████████████████| 26.4M/26.4M [01:00<00:00, 433kB/s]
100%|███████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 103kB/s]
100%|██████████████████████████████████████████████████| 4.42M/4.42M [00:02<00:00, 2.05MB/s]
100%|██████████████████████████████████████████████████| 5.15k/5.15k [00:00<00:00, 2.62MB/s]


In [3]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of your: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of your: torch.Size([64]) torch.int64


## 모델 만들기

In [4]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻기
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 매개변수 최적화하기

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)
    
    # 역전파
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n----------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.306786  [   64/60000]
loss: 2.291826  [ 6464/60000]
loss: 2.267572  [12864/60000]
loss: 2.259508  [19264/60000]
loss: 2.246582  [25664/60000]
loss: 2.220053  [32064/60000]
loss: 2.225921  [38464/60000]
loss: 2.190755  [44864/60000]
loss: 2.193236  [51264/60000]
loss: 2.165634  [57664/60000]
Test Error: 
 Accuracy: 45.0%, Avg loss: 2.152248 

Epoch 2
----------------------------
loss: 2.167120  [   64/60000]
loss: 2.149762  [ 6464/60000]
loss: 2.089551  [12864/60000]
loss: 2.105824  [19264/60000]
loss: 2.044442  [25664/60000]
loss: 1.999691  [32064/60000]
loss: 2.021861  [38464/60000]
loss: 1.940198  [44864/60000]
loss: 1.950315  [51264/60000]
loss: 1.883197  [57664/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.874650 

Epoch 3
----------------------------
loss: 1.909198  [   64/60000]
loss: 1.870521  [ 6464/60000]
loss: 1.754718  [12864/60000]
loss: 1.795866  [19264/60000]
loss: 1.680439  [25664/60000]
loss: 1.650682  [32064/60000]
loss:

## 모델 저장하기

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
classes = [
  "T-shirt/top",
  "Trouser",
  "Pullover",
  "Dress",
  "Coat",
  "Sandal",
  "Shirt",
  "Sneaker",
  "Bag",
  "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
